# GTFS
## Savivaldybės
VINTRA viešai skelbia [GTFS failus](https://www.visimarsrutai.lt/gtfs/), kurie apima viešojo transporto stoteles, maršrutus, tvarkaraščius ir kitą pagal GTFS specifikaciją apibrėžtą informaciją. Toliau pateikiama šių failų analizė.

```{admonition} VINTRA pateikti ne visų savivaldybių GTFS failai
:class: warning
VINTRA pateikti 57 iš 60 savivaldybių GTFS duomenų failai.

Trūksta: Telšių rajono, Šakių rajono, Širvintų rajono savivaldybių GTFS failų.
```

### Teisinis teikimo pagrindas
> 1.  Viešojo transporto kelionių duomenų kaupimo tvarkos aprašas (toliau – Aprašas) nustato viešojo transporto kelionių duomenų (toliau – duomenys) teikimo į Viešojo transporto kelionių duomenų informacinę sistemą (toliau – IS „Vintra“) tvarką, pagal kurią IS „Vintra“ duomenų teikėjai **privalo** teikti viešojo transporto kelionių duomenis.{cite}`sumin_vintra_duomenu_kaupimo_tvarka`


Duomenis į IS „Vintra“ teikia{cite}`sumin_vintra_duomenu_kaupimo_tvarka`:
1. Lietuvos automobilių kelių direkcija prie Susisiekimo ministerijos;
2. Lietuvos transporto saugos administracija, valdanti ir organizuojanti keleivių vežimą tolimojo ir tarptautinio reguliariojo susisiekimo maršrutais;
3. Savivaldybių institucijos arba jų įgaliotos įstaigos, valdančios ir organizuojančios keleivių vežimą vietinio (miesto ar priemiestinio) reguliariojo susisiekimo maršrutais;
4. Oro ir jūrų uostus valdančios įmonės;
5. Vežėjai, teikiantys keleivių vežimo viešuoju geležinkelių transportu paslaugas;
6. Vežėjai, teikiantys keleivių vežimo viešuoju jūrų ir vidaus vandenų transportu paslaugas;
7. Vežėjai, teikiantys keleivių vežimo reguliariojo susisiekimo kelių transporto maršrutais paslaugas ir turintys atitinkamą paslaugų teikimo licenciją arba leidimą.


Savivaldybių institucijos arba jų įgaliotos įstaigos, valdančios ir organizuojančios keleivių vežimą vietinio (miesto ar priemiestinio) reguliariojo susisiekimo maršrutais, teikia šiuos duomenis{cite}`sumin_vintra_duomenu_kaupimo_tvarka`:
1. autobusų stočių, stotelių ir kitų maršrutų punktų;
2. maršruto trasos trajektorijos;
3. maršruto;
4. reiso;
5. tvarkaraščio;
6. transporto priemonių geografinės padėties;
7. tarifų ir kainoraščių;
8. vežėjų.


### Privalomų failų nebuvimas
Pagal [specifikaciją](https://gtfs.org/schedule/reference/#dataset-files) GTFS privalo sudaryti bent penki failai (`agency.txt`, `stops.txt`, `routes.txt`, `trips.txt`, `stop_times.txt`).

Toliau žemėlapyje pateikiamos savivaldybės, kurios neturi bent vieno iš privalomų failų.

```{admonition} Trūksta bent vieno pagal GTFS specifikaciją privalomo failo
:class: warning

Tarp VINTRA pateiktų GTFS, 10 savivaldybių trūksta bent vieno pagal GTFS specifikaciją privalomo failo.
```

In [ ]:
import datetime
import json
import os
from zipfile import ZipFile

import pandas as pd
import plotly.express as px

working_directory = f'{os.getcwd()}/../data/saltiniai/vintra/'
gtfs_files_directory = os.path.join(working_directory, 'gtfs')
netex_directory = os.path.join(working_directory, 'netex')

mapbox_access_token = open("../.mapbox_token").read()
px.set_mapbox_access_token(mapbox_access_token)

lithuania_center = {'lat': 55.169438, 'lon': 23.881275}

municipalities_gtfs_file_mapping = pd.read_csv(os.path.join(working_directory, 'vintra-gtfs-file-mapping.csv'),
                                               na_filter=False)
vintra_netex_mapping = pd.read_csv(os.path.join(working_directory, 'vintra-netex-file-mapping.csv'), na_filter=False)

with open('../data/geojson/municipalities.geojson', 'r') as municipalities_geojson_file:
    municipalities_geojson = json.load(municipalities_geojson_file)

In [ ]:
# Uncomment to update GTFS files
# import requests
#
# vintra_gtfs_df = pd.read_csv(os.path.join(working_directory, 'vintra-gtfs-files.csv'))
#
# for _, row in vintra_gtfs_df.iterrows():
#     file_name = row['File']
#     if file_name.endswith('.zip'):
#         url = f'https://www.visimarsrutai.lt/gtfs/{file_name}'
#
#         response = requests.get(url, stream=True)
#
#         with open(os.path.join(gtfs_files_directory, file_name), "wb") as handle:
#             for data in response.iter_content(chunk_size=8192):
#                 handle.write(data)
#
#
# for _, row in vintra_netex_mapping.iterrows():
#     file_name = row['Failas']
#     if file_name.endswith('.zip'):
#         url = f'https://www.visimarsrutai.lt/netex/{file_name}'
#
#         response = requests.get(url, stream=True)
#
#         with open(os.path.join(netex_directory, file_name), "wb") as handle:
#             for data in response.iter_content(chunk_size=8192):
#                 handle.write(data)
# gtfs_file_stats_df = pd.DataFrame()
#
# for file in sorted(os.listdir(gtfs_files_directory)):
#     if file.endswith('.zip'):
#         filename, _, _ = file.partition('.zip')
#
#         p = subprocess.Popen([
#             f'java -jar gtfs-validator-301.jar -i gtfs/{file} -o reports -v {filename}_report.json -e {filename}_system_errors.json -n -c lt'],
#             cwd=working_directory, shell=True, stdout=subprocess.PIPE,
#             stderr=subprocess.PIPE)
#         out, err = p.communicate(timeout=60)
#         errcode = p.returncode
#
#         _, _, gtfs_files_txt = out.decode("utf-8").partition('seconds\n')
#         gtfs_files = gtfs_files_txt.splitlines()
#
#         gtfs_files_dict = {'failas': filename}
#         for gtfs_file_rep in gtfs_files:
#             gtfs_file, c = gtfs_file_rep.split('\t')
#             gtfs_files_dict[gtfs_file] = c if c != 'MISSING_FILE' else None
#
#         gtfs_file_stats_df = gtfs_file_stats_df.append(gtfs_files_dict, ignore_index=True, )
#
# gtfs_file_stats_df = gtfs_file_stats_df.reindex(
#     columns=[
#         'failas',
#         'agency.txt',
#         'calendar.txt',
#         'calendar_dates.txt',
#         'routes.txt',
#         'shapes.txt',
#         'stop_times.txt',
#         'stops.txt',
#         'trips.txt',
#         'fare_attributes.txt',
#         'fare_rules.txt',
#         'attributions.txt',
#         'feed_info.txt',
#         'frequencies.txt',
#         'levels.txt',
#         'pathways.txt',
#         'transfers.txt',
#         'translations.txt'
#     ]
# ).set_index('failas')
#
# gtfs_file_stats_df[
#     [
#         'agency.txt',
#         'calendar.txt',
#         'calendar_dates.txt',
#         'routes.txt',
#         'shapes.txt',
#         'stop_times.txt',
#         'stops.txt',
#         'trips.txt',
#         'fare_attributes.txt',
#         'fare_rules.txt',
#     ]
# ] = gtfs_file_stats_df[
#     [
#         'agency.txt',
#         'calendar.txt',
#         'calendar_dates.txt',
#         'routes.txt',
#         'shapes.txt',
#         'stop_times.txt',
#         'stops.txt',
#         'trips.txt',
#         'fare_attributes.txt',
#         'fare_rules.txt',
#     ]
# ].fillna('❌')
#
# gtfs_file_stats_df.fillna('⚠️', inplace=True)
# gtfs_file_stats_df.style.set_sticky(axis="index")
#
# gtfs_file_stats_df

In [ ]:
import numpy as np
import json

reports_dir = f'{working_directory}/gtfs-reports/'

gtfs_notices_df = pd.DataFrame()

for file in sorted(os.listdir(reports_dir)):
    if file.endswith('report.json'):
        gtfs_filename, _, _ = file.partition('_report.json')

        with open(os.path.join(reports_dir, file)) as fp:
            data = json.load(fp)

            for notice in data['notices']:
                gtfs_notice_df = pd.DataFrame(
                    [[f'{gtfs_filename}.zip', notice['code'], notice['severity'], notice['totalNotices']]],
                    columns=['Failas', 'Klaida', 'Sunkumas', 'Viso']
                )
                gtfs_notices_df = pd.concat([gtfs_notices_df, gtfs_notice_df])

gtfs_notices_df['Viso'] = pd.to_numeric(gtfs_notices_df['Viso'], downcast='integer')

gtfs_errors_df = gtfs_notices_df[gtfs_notices_df['Sunkumas'] == 'ERROR'].drop(columns=['Sunkumas'])

gtfs_errors_df = gtfs_errors_df.pivot_table(index='Failas', columns='Klaida', values='Viso', aggfunc='sum',
                                            margins=True, fill_value=0)

gtfs_notices_missing_required_file = gtfs_notices_df[gtfs_notices_df['Klaida'] == 'missing_required_file']

municipalities_with_some_missing_required_file = municipalities_gtfs_file_mapping.merge(
    gtfs_notices_missing_required_file,
    on='Failas',
    how='left'
)[['Savivaldybe', 'Failas', 'Klaida']]

municipalities_with_some_missing_required_file = municipalities_with_some_missing_required_file.replace(
    {'Klaida': {
        'missing_required_file': 1,
        np.nan: 0}
    }
)

fig = px.choropleth(
    municipalities_with_some_missing_required_file,
    geojson=municipalities_geojson,
    locations="Savivaldybe",
    featureidkey="properties.name",
    color='Klaida',
    center=lithuania_center,
    fitbounds='locations',
    basemap_visible=False,
    color_continuous_scale='rdylgn_r',
    projection="mercator",
    title='Savivaldybės turinčios privalomus GTFS failus VINTRA (pažymėtos žaliai)',
)
fig.update_layout(margin={"r": 0, "l": 0, "b": 0, "t": 32})
fig.update_coloraxes(showscale=False)
fig.show()

## Duomenų rinkinio failai

In [ ]:
from zipfile import BadZipFile


def has_non_empty_gtfs_file(file: str, gtfs_file: str) -> bool:
    if not file:
        return False
    with ZipFile(os.path.join(gtfs_files_directory, file)) as gtfs_zip:
        gtfs_zip_files = gtfs_zip.namelist()

        if gtfs_file not in gtfs_zip_files:
            return False

        try:
            with gtfs_zip.open(gtfs_file) as zip_file:
                return pd.read_csv(zip_file).shape[0] > 0
        except BadZipFile:
            return False


gtfs_files_by_specification = [
    'agency.txt',
    'stops.txt',
    'routes.txt',
    'trips.txt',
    'stop_times.txt',
    'calendar.txt',
    'calendar_dates.txt',
    'shapes.txt',
    'feed_info.txt',
    'fare_rules.txt',
    'fare_attributes.txt',
    'frequencies.txt',
    'transfers.txt',
    'pathways.txt',
    'levels.txt',
    'translations.txt',
    'attributions.txt',
]

municipalities_files_availability = municipalities_gtfs_file_mapping.copy()

for gtfs_file_by_specification in gtfs_files_by_specification:
    municipalities_files_availability[gtfs_file_by_specification] = municipalities_files_availability.apply(
        lambda m: has_non_empty_gtfs_file(m['Failas'], gtfs_file_by_specification), axis=1).astype(int)

melted_municipalities_files_availability = municipalities_files_availability.melt(
    id_vars=['Savivaldybe'],
    value_vars=gtfs_files_by_specification,
    var_name='GTFS Failas',
    value_name='has_file',
    ignore_index=True,
)

fig = px.choropleth(
    melted_municipalities_files_availability,
    geojson=municipalities_geojson,
    locations="Savivaldybe",
    featureidkey="properties.name",
    color='has_file',
    color_continuous_scale='rdylgn',
    fitbounds='locations',
    basemap_visible=False,
    projection="mercator",
    facet_col='GTFS Failas',
    facet_col_wrap=3,
    facet_row_spacing=0,
    facet_col_spacing=0,
    title='VINTRA savivaldybių duomenų rinkinio failai (žalia spalva - failas prieinamas)',
    height=2500,
)
fig.update_layout(margin={"r": 0, "t": 32, "l": 0, "b": 0})
fig.update_coloraxes(showscale=False)

fig.show()

## Maršrutai be galiojančių reisų

Kiekvienas maršrutas susideda ir vieno ar keleto reisų, kurie turi galiojimo laiką. Kai duomenys ilgą laiką natnaujinami reisai nustoja galioti, kol maršrutai lieka visiškai be jokių galiojančių reisų. Toliau pateikiama kokia procentinė maršrutų dalis nebeturi nė vieno galiojančio reiso.

```{admonition} Nė vieno galiojančio maršruto
:class: warning
23 savivaldybės neturi nė vieno galiojančio maršruto. Tai reikia, kad duomenys šiose savivaldybėse nėra atnaujinami.
```

```{admonition} Maršrutai be galiojančių reisų
:class: warning
29 savivaldybėse visi maršrutai turi bent vieną galiojantį reisą. Tai reiškia, kad likusiose savivaldybėse duomenys galimai yra neatnaujinami.
```

In [ ]:
from datetime import date
from typing import Optional

municipalities_valid_routes_df = municipalities_gtfs_file_mapping.copy()


def calculate_valid_routes(file: str) -> float:
    if not file:
        return 0
    with ZipFile(os.path.join(gtfs_files_directory, file)) as gtfs_zip:
        gtfs_zip_files = gtfs_zip.namelist()
        if "calendar.txt" not in gtfs_zip_files or "routes.txt" not in gtfs_zip_files or "trips.txt" not in gtfs_zip_files:
            return 0

        routes_df = pd.read_csv(gtfs_zip.open("routes.txt"))
        calendar_df = pd.read_csv(gtfs_zip.open("calendar.txt"), parse_dates=['start_date', 'end_date'])
        trips_df = pd.read_csv(gtfs_zip.open("trips.txt"))

        merged_df = trips_df.merge(calendar_df, on='service_id')

        total_routes = routes_df['route_id'].nunique()
        if total_routes == 0:
            return 0

        valid_routes = merged_df[(merged_df['end_date'] >= '2022-04-08')]['route_id'].nunique()

        return (valid_routes / total_routes) * 100


def calculate_last_valid_trip_date_formatted(file: str) -> Optional[str]:
    if not file:
        return '-'
    with ZipFile(os.path.join(gtfs_files_directory, file)) as gtfs_zip:
        gtfs_zip_files = gtfs_zip.namelist()
        if "calendar.txt" not in gtfs_zip_files:
            return '-'

        calendar_df = pd.read_csv(gtfs_zip.open("calendar.txt"), parse_dates=['start_date', 'end_date'])
        max_end_date = calendar_df['end_date'].max()

        return str(max_end_date.date()) if max_end_date else '-'


municipalities_valid_routes_df['Dalis, %'] = municipalities_valid_routes_df.apply(
    lambda m: calculate_valid_routes(m['Failas']), axis=1)

municipalities_valid_routes_df['Paskutinis galiojantis reisas'] = municipalities_valid_routes_df.apply(
    lambda m: calculate_last_valid_trip_date_formatted(m['Failas']), axis=1)

fig = px.choropleth(
    municipalities_valid_routes_df,
    geojson=municipalities_geojson,
    locations="Savivaldybe",
    featureidkey="properties.name",
    color='Dalis, %',
    color_continuous_scale='rdylgn',
    fitbounds='locations',
    basemap_visible=False,
    projection="mercator",
    title='VINTRA galiojančių maršrutų dalis savivaldybėse (žaliau geriau)',
    hover_data=['Paskutinis galiojantis reisas']
)
fig.update_layout(margin={"r": 0, "t": 32, "l": 0, "b": 0})

fig.show()

In [ ]:
# Experiment without px.choropleth
# from plotly.graph_objs.choropleth import ColorBar
# import plotly.graph_objects as go
#
# fig = go.Figure(
#     data=go.Choropleth(
#         geojson=municipalities_geojson,
#         locations=municipalities_valid_routes_df["Savivaldybe"],
#         featureidkey="properties.name",
#         z=municipalities_valid_routes_df['Dalis, %'],
#         colorscale='reds',
#         colorbar=ColorBar(
#             ticksuffix='%',
#             showticksuffix='last',
#         ),
#     ),
# )
#
# fig.update_layout(
#     title_text='Maršrutų be galiojančių reisų dalis VINTRA sistemoje',
#     geo=dict(
#         showframe=False,
#         showcoastlines=False,
#         visible=False,
#         projection_type='mercator',
#         fitbounds='locations',
#     ),
#     legend=dict(
#         orientation="h",
#         yanchor="bottom",
#         y=1.02,
#         xanchor="right",
#         x=1
#     ),
#     margin={"r": 0, "t": 32, "l": 0, "b": 0}
# )
#
# fig.show()


## GTFS patikrinimas
### GTFS patikrinimo klaidos

In [ ]:
# def show_notices_table_by_severity(severity: str) -> pd.DataFrame:
#     gtfs_errors_df = gtfs_notices_df[gtfs_notices_df['sunkumas'] == severity].drop(columns=['sunkumas'])
#
#     gtfs_errors_df = gtfs_errors_df.pivot_table(index='failas', columns='klaida', values='viso', aggfunc='sum',
#                                                 margins=True, fill_value=0)
#
#     gtfs_errors_df.style.set_sticky(axis="index")
#     gtfs_errors_df = gtfs_errors_df.style.apply(lambda x: ["background: orange" if v > 0 else '' for v in x], axis=1)
#
#     return gtfs_errors_df
#
#
# show_notices_table_by_severity('ERROR')

### GTFS patikrinimo įspėjimai

In [ ]:
# show_notices_table_by_severity('WARNING')

### Stotelės

In [ ]:

import plotly.express as px

all_stops = pd.DataFrame()
for file in sorted(os.listdir(gtfs_files_directory)):
    if file.endswith('.zip') and file != 'gtfs_all.zip':
        filename, _, _ = file.partition('.zip')

        with ZipFile(os.path.join(gtfs_files_directory, file)) as gtfs_zip:
            if "stops.txt" not in gtfs_zip.namelist():
                continue

            stops_csv = gtfs_zip.open("stops.txt")

        stops_df = pd.read_csv(stops_csv)
        stops_df['failas'] = filename
        all_stops = pd.concat([all_stops, stops_df])

mapbox_access_token = open("../.mapbox_token").read()
px.set_mapbox_access_token(mapbox_access_token)

fig = px.scatter_mapbox(
    data_frame=all_stops,
    lat='stop_lat',
    lon='stop_lon',
    mapbox_style="light",
    zoom=6,
    title='Stotelės',
    hover_name='stop_name',
    color='failas',
)

fig.update_layout(
    mapbox_layers=[
        {
            "sourceattribution": '© <a href="https://judumas.vycius.lt" target="_blank">Karolis Vyčius</a> © <a href="https://www.visimarsrutai.lt/gtfs/" target="_blank">Visimarsrutai.lt</a>'
        }
    ])
fig.update_layout(margin={"r": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
with ZipFile(os.path.join(gtfs_files_directory, 'google_transit.zip')) as gtfs_zip:
    stops_csv = gtfs_zip.open("stops.txt")

    google_transit_vintra_stops_df = pd.read_csv(stops_csv)

    fig = px.scatter_mapbox(
        data_frame=google_transit_vintra_stops_df,
        lat='stop_lat',
        lon='stop_lon',
        mapbox_style="light",
        zoom=6,
        title='Google Maps stotelės iš Vintra',
        hover_name='stop_name',
    )

    fig.update_layout(
        mapbox_layers=[
            {
                "sourceattribution": '© <a href="https://judumas.vycius.lt" target="_blank">Karolis Vyčius</a> © <a href="https://www.visimarsrutai.lt/gtfs/" target="_blank">Visimarsrutai.lt</a>'
            }
        ])
    fig.update_layout(margin={"r": 0, "l": 0, "b": 0})
    fig.show()

## Tolimasis susisiekimas
### Teisinis duomenų teikimo pagrindas

> 8. Vežėjas įsipareigoja:
> 8.9. teikti viešojo transporto kelionių duomenis, nurodytus Viešojo transporto kelionių duomenų kaupimo tvarkos aprašo 15 punkte, į Viešojo transporto kelionių duomenų informacinę sistemą (IS „Vintra“);{cite}`ltsa_vezeju_prievole_teikti`

Lietuvos transporto saugos administracija teikia šiuos tolimojo ir tarptautinio reguliariojo susisiekimo autobusais maršrutų duomenis{cite}`sumin_vintra_duomenu_kaupimo_tvarka`:
1. autobusų stočių, stotelių ir kitų maršrutų punktų;
2. maršruto trasos trajektorijos;
3. maršruto;
4. reiso;
5. tvarkaraščio;
6. vežėjų;
7. transporto priemonių geografinės padėties;
8. tarifų ir kainoraščių.

Vežėjai, teikiantys keleivių vežimo tolimojo ir tarptautinio reguliariojo susisiekimo maršrutais paslaugas, teikia šiuos duomenis{cite}`sumin_vintra_duomenu_kaupimo_tvarka`:
15.1.  transporto priemonių geografinės padėties;
15.2.  tarifų ir kainoraščių;




In [ ]:
gtfs_notices_df[gtfs_notices_df['Failas'] == 'LTSAR.zip']

In [ ]:
def calculate_invalid_routes_debug(file: str) -> float:
    if not file:
        return 100
    with ZipFile(os.path.join(gtfs_files_directory, file)) as gtfs_zip:
        gtfs_zip_files = gtfs_zip.namelist()
        print('')
        if "calendar.txt" not in gtfs_zip_files or "routes.txt" not in gtfs_zip_files or "trips.txt" not in gtfs_zip_files:
            return 100

        routes_df = pd.read_csv(gtfs_zip.open("routes.txt"))
        calendar_df = pd.read_csv(gtfs_zip.open("calendar.txt"), parse_dates=['start_date', 'end_date'])
        trips_df = pd.read_csv(gtfs_zip.open("trips.txt"))

        merged_df = trips_df.merge(calendar_df, on='service_id')

        total_routes = routes_df['route_id'].nunique()
        if total_routes == 0:
            return 100

        valid_routes = merged_df[(merged_df['end_date'] >= '2022-04-08')]['route_id'].nunique()

        return (1 - valid_routes / total_routes) * 100


with ZipFile(os.path.join(gtfs_files_directory, 'SiauliuM.zip')) as gtfs_zip:
    df = pd.read_csv(gtfs_zip.open('calendar.txt'), parse_dates=['start_date', 'end_date'])

    print(df.head().sort_values('end_date', ascending=False))

# print(calculate_invalid_routes_debug('PanevezioM.zip'))

In [ ]:
with ZipFile(os.path.join(gtfs_files_directory, 'LTSAR.zip')) as gtfs_zip:
    ltsa_stops_df = pd.read_csv(gtfs_zip.open("stops.txt"))

    fig = px.scatter_mapbox(
        data_frame=ltsa_stops_df,
        lat='stop_lat',
        lon='stop_lon',
        mapbox_style="light",
        zoom=6,
        title='LTSA',
        hover_name='stop_name',
    )

    fig.update_layout(
        mapbox_layers=[
            {
                "sourceattribution": '© <a href="https://judumas.vycius.lt" target="_blank">Karolis Vyčius</a> © <a href="https://www.visimarsrutai.lt/gtfs/" target="_blank">Visimarsrutai.lt</a>'
            }
        ])
    fig.update_layout(margin={"r": 0, "l": 0, "b": 0})
    fig.show()